# Matrices

Matrices are, fundamentally, about moving rigid objects around in space. This tutorial builds a couple matrices (in 2D), then uses them to plot a circle in different positions.

The major tricky part of this is that the object - the circle - is ALSO represented as a matrix (a **3xn** matrix, **n** being the number of points used to represent the circle).

Some other tricky parts with matrices

- Order matters. Doing m1 then m2 is NOT the same as m2 then m1. We will use the convention that **pts_new = m1 @ m2 @ m3 pts_orig** means multiply pts_orig by m3, then m2, then m1. Yes, that feels backwards. But that is the convention (left multiply) in most software that uses matrices

- You'll notice I used @ and not *. numpy decided to use @ instead of * for matrix multiply ( * means item by item multiplication a) will only work if the two matrics are the same size and b) probably isn't what you want).

This example puts all of the code that will be re-used into functions in a Python file (**matrix_routines.py**), rather than just writing it in-line. So you should open up that file and look at it, too, then come back and work through this tutorial. 

In this tutorial, we will
- Multiply matrices together
- Use matrices to place objects in the world

A reminder that all matrices are 3x3 (even though we are in 2d) so that we can do translations (the upper left is the 2x2 matrix)

First, look at matrix_routines.py. Look for code that:
- Creates matrices
- Creates a circle as a list of points
- Multiplies points by a matrix and plots the before and after
-- Where are the points made/created?
-- Where does the multiplicaiton happen?

There is an example here that shows swapping the order of the matrices. Try drawing what the rotations/translations would look like on a piece of paper and try to match up the transformations with the code.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# All of the matrix routines are in here - we can import the Python (.py) file just like the imports above
#  All of the functions in there will be accessed as mr.
import matrix_routines as mr


# These commands will force JN to actually re-load the external file when you re-execute the import command
%load_ext autoreload
%autoreload 2

### Matrices and points

Matrices pick "objects" up and move them around. What do those objects look like? They're just a list of points. Here we make an object - a triangle - and perform a couple of transformations on it

In [ ]:
# It's easier to start with a numpy array that has ones in it to make the points because you'll need ones in the 3rd row
n_points = 3
pts_triangle = np.ones((3, n_points))
# Make a right-sided triangle with the center of the origin in the middle
pts_triangle[0:2, 0] = [-2.0, 1.25]  # Upper left corner
pts_triangle[0:2, 1] = [2.0, -0.5]   # Pointy bit at the right
pts_triangle[0:2, 2] = [-2.0, -0.5]   # 90 degree angle part

print(pts_triangle)

In [ ]:
# Now make a matrix to multiply the points by - this matrix translates the points up and to the right by 1.5, 2.5
mat_trans = mr.make_translation_matrix(1.5, 2.5)

# THis is a translation matrix - ones along the diagonal, the the dx, dy in the top right
print(mat_trans)

In [ ]:
# How to get the points translated by the matrix - notice that pts is on the right and we use @
#  3x3 matrix times a 3xn matrix yields a 3xn matrix
pts_triangle_moved = mat_trans @ pts_triangle

### Make some useful plotting functions

In [ ]:
def plot_axes_and_big_box(axs):
    """Plot the origin and x,y axes with a box at -5, -5 to 5, 5
    @param axs - figure axes"""

    # Put a black + at the origin
    axs.plot(0, 0, '+k')
    # Draw one red arrow for the x axis (x,y, dx, dy)
    axs.arrow(x=0, y=0, dx=1, dy=0, color='red')
    # Draw a blue arrow for the y axis
    axs.arrow(x=0, y=0, dx=0, dy=1, color='blue')

    # Draw a box around the world to make sure the plots stay the same size
    axs.plot([-5, 5, 5, -5, -5], [-5, -5, 5, 5, -5], '-k')

    # This makes sure the x and y axes are scaled the same
    axs.axis('equal')




In [ ]:
def plot_transformed_axes(axs, mat):
    """Plot where the coordinate system (0,0 and x,y axes) goes to when transformed by mat
    @param axs - figure axes
    @param mat - the matrix"""

    # Moved coordinate system - draw the moved coordinate system and axes
    #  The x axis as a vector - notice that the 3rd coordinate is a zero because vectors can't translate
    x_axis = np.array([1, 0, 0]).transpose()
    x_axis_moved = mat @ x_axis

    y_axis = np.array([0, 1, 0]).transpose()
    y_axis_moved = mat @ y_axis

    # The origin, however, is a point and it has a 1 in that third column
    origin = np.array([0, 0, 1]).transpose()

    origin_moved = mat @ origin

    # Put a blue X at the placd the origin moved to
    axs.plot(origin_moved[0], origin_moved[1], 'Xb', markersize=5)
    # Draw an arrow from there to the end of the moved x axis
    axs.arrow(x=origin_moved[0], y=origin_moved[1], dx=x_axis_moved[0], dy=x_axis_moved[1], color='red', linestyle="--")
    # Draw a blue arrow for the y axis
    axs.arrow(x=origin_moved[0], y=origin_moved[1], dx=y_axis_moved[0], dy=y_axis_moved[1], color='blue', linestyle="--")


In [ ]:
def plot_pts(axs, pts, fmt='-k'):
    """ plot the points in the window
    @param axs - the window to draw into
    @param pts - the 3xn array of points
    @param fmt - optional format parameter"""

    # This gets the x values (in row 0) and the y values and just does a regular plot
    axs.plot(pts[0, :], pts[1, :], fmt)

    pts_close = np.zeros((2, 2))
    pts_close[:, 0] = pts[0:2, 0]
    pts_close[:, 1] = pts[0:2, -1]
    # and close the polygon
    axs.plot(pts_close[0, :], pts_close[1, :], fmt)


#### Plot the triangle without any transform on the left, and the translate on the right

TODO: look at the plotting code and make sure you understand the format of the points numpy array

TODO: Notice how the origin and x and y axis are transformed

TODO: Change the matrix to a scale matrix, then a rotation matrix to see what those do

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

# No transformation on the left
plot_axes_and_big_box(axs[0])
plot_pts(axs[0], pts_triangle)
axs[0].set_title("No transform")

# Transform
plot_axes_and_big_box(axs[1])
# Show how the axes moved
plot_transformed_axes(axs[1], mat_trans)
plot_pts(axs[1], pts_triangle_moved, fmt=":k")
axs[1].set_title("Translated")

## Example: What happens if you swap the order of two matrix multiplications? 

In this example we'll do a translate and a rotate in the left window, and a rotate translate in the middoe window, then a scale, rotate, translate in the right window. 

In [ ]:
""" Make the plot for rotation-translation versus translation-rotation"""
# Make the plot that shows the difference between rotate-translate and translate-rotate
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Rotate the object then translate it - notice that the rotation goes on the right
mat_rot_trans = mr.make_translation_matrix(1, 2) @ mr.make_rotation_matrix(np.pi / 4.0)   # pts go here

axs[0].set_title("Rot trans")
plot_axes_and_big_box(axs[0])
pts_triangle_rot_trans = mat_rot_trans @ pts_triangle
plot_transformed_axes(axs[0], mat_rot_trans)
plot_pts(axs[0], pts_triangle_rot_trans, ":k")


# Reverse the order of operations
# Now create the matrix in the reverse order - try to predict what this will look like
#   Set mat to be a translation, rotation matrix (same params as above)
axs[1].set_title("Trans rot")

# Translate first, then rotate
mat_trans_rot = mr.make_rotation_matrix(np.pi / 4.0) @ mr.make_translation_matrix(1, 2)   # pts go here

plot_axes_and_big_box(axs[1])
pts_triangle_trans_rot = mat_trans_rot @ pts_triangle
plot_transformed_axes(axs[1], mat_trans_rot)
plot_pts(axs[1], pts_triangle_trans_rot, ":k")


# Last one - scale first, then rotate, then translate
# Now do a matrix that is a scale 0.5,2.0, rotate pi/4, translate (1,2)
mat_scl_rot_trans = mr.make_translation_matrix(1, 2) @ \
                    mr.make_rotation_matrix(np.pi / 4.0) @ \
                    mr.make_scale_matrix(0.5, 2.0)   # pts go here


axs[2].set_title("Scale, rotate, translate")
plot_axes_and_big_box(axs[2])
pts_triangle_scl_rot_trans = mat_scl_rot_trans @ pts_triangle
plot_transformed_axes(axs[2], mat_scl_rot_trans)
plot_pts(axs[2], pts_triangle_scl_rot_trans, ":k")
